# Training Spacey
### Timbuktu Chronicles
#### This file was created and used on Google Colab. The link to view the most updated version is: https://colab.research.google.com/drive/1H1Y8LUTKYfBvy8GUm8rOXLQL-AKVhieH?usp=sharing

Some linkes regarding the training data:
- https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
- https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/ 
- https://www.youtube.com/watch?v=YBRF7tq1V-Q 
  - and the corresponding code file: https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_02_customizing_spacy.py

To-do:
- See how to get the output of the spacey NER into a file so you can edit it 
(might have to do with the save_data function in the code for the video above - and also for the code in the lesson before it, 4.1)
- Correct the spacey output
- Use the corrected output to upload back to Spacey and use for training

Refer to TimbuktuChronicles Google Colab notebook for some initial examples with Spacy and file access.

Some segments of my code was based off of the YouTube video https://www.youtube.com/watch?v=YBRF7tq1V-Q , and corresponding code file: https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_02_customizing_spacy.py


In [28]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
import json

In [37]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [14]:
file1 = open("/content/gdrive/My Drive/fattashEnglish.txt", "r")
text = file1.read()
#sample

In [15]:
doc1 = nlp(text)
#doc1
#doc1.ents - all the entities in the doc
# pprint([(X, X.ent_iob_, X.ent_type_) for X in doc1]) list like: [(In, 'O', ''),(Muhammad, 'B', 'PERSON'), etc]


In [23]:
def test_model(text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) > 0:
        results = [text, {"entities": entities}]
        return (results)

In [24]:
TRAIN_DATA = []
file1 = open("/content/gdrive/My Drive/fattashEnglish.txt", "r")

for line in file1.readlines():
  #print(line.strip())
   line = line.strip()
   results = test_model(line)

   if results != None:
     TRAIN_DATA.append(results)

#TRAIN_DATA

In [26]:
def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [29]:
save_data("/content/gdrive/My Drive/fattash_training_data.json", TRAIN_DATA)

### To-do:
- correct the output JSON file
- train the model 
  - see part 5 here: https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/ 
  - and https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_03_customizing_spacy.py

  Code for training the model is based off of the following 2 articles:
  - https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/
  - https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

In [30]:
# load the training data back from the JSON file, via https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_03_customizing_spacy.py
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)


In [33]:
TRAIN_DATA_reloaded = load_data("/content/gdrive/My Drive/fattash_training_data.json")
#TRAIN_DATA_reloaded

In [ ]:
# training the model, via part 5 here:  https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/


# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

In [38]:
# via https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# load a spacy model
nlp = spacy.blank('en')  

# create ner pipeline:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

# add the labels from TRAIN_DATA into the ner:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

#we only want to train NER, so this code tells it to ignore training everything else
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(30): # customize the number of iterations
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses) # not sure why I have this 100% - 5% pattern or why the numbers following 'ner' are so high. Maybe I should start with a different spacy model for nlp?

  5%|▍         | 3/65 [00:00<00:02, 29.63it/s]

{'ner': 331.21830189854023}


  5%|▍         | 3/65 [00:00<00:02, 28.11it/s]

{'ner': 176.8498071495291}


  5%|▍         | 3/65 [00:00<00:02, 26.88it/s]

...

{'ner': 126.10233196985612}


  5%|▍         | 3/65 [00:00<00:02, 23.15it/s]

{'ner': 108.72646844654555}


100%|██████████| 65/65 [00:02<00:00, 24.49it/s]

{'ner': 105.88652681170483}


In [42]:
# test the trained model: 
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    #print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

# might need to split the TRAIN_DATA into TRAIN and TEST parts?

# testing the model on something it hasn't seen before:
doc = nlp("Askia Muhammad was the Khalifa of the Songhay Empire in the fifteenth/sixteenth century.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Songhay', 'GPE')]


In [ ]:
# saving/reloading the model so you don't have to train each time, via https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# saving the model:
"""
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
"""

# reloading the model
# model = spacy.load('model_name')

In [ ]:
# visuals
# displacy.render(doc, jupyter=True, style='ent') - coloring the entities
# displacy.render(doc, style='dep', jupyter = True, options = {'distance': 120}) - graph